Here is a sample code for gru4rec using recbole. 
I made it based on the following code from a past competition
https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

I think you can get a better score if you change the training data or epoch size, etc. Enjoy!

In [1]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install recbole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!unzip sessions.zip

Archive:  sessions.zip
replace sessions.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
!pip install hyperopt==0.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tqdm
import polars as pl
import numpy as np
import pandas as pd
import seaborn as sns
import random
import os 
import h5py
import sys
import gc

from matplotlib import pyplot as plt
import pyarrow.parquet as pq

# 1. Przygotowanie danych

In [7]:
sessions_df = pd.read_json('sessions.jsonl')
sessions_df

,session_id,timestamp,user_id,track_id,event_type
0,124,2022-07-19 17:17:13.000,101,7b7AziuX7EeIzvatDcC0RS,play
1,124,2022-07-19 17:20:33.880,101,5RubKOuDoPn5Kj5TLVxSxY,play
2,124,2022-07-19 17:22:48.894,101,23zzKOubAzAu8xlQdkVvZ2,play
3,124,2022-07-19 17:28:23.521,101,2DVkYfl2YuOMlRfC98srEG,play
4,124,2022-07-19 17:32:25.619,101,5wTuWXWREngkFGtBXhOaI9,play
...,...,...,...,...,...
1279798,60198,2022-10-19 02:22:52.276,4100,3tSuRiNgJmMqmYR23dCqnR,play
1279799,60198,2022-10-19 02:23:40.099,4100,3tSuRiNgJmMqmYR23dCqnR,like
1279800,60198,2022-10-19 02:26:35.676,4100,None,advertisment
1279801,60198,2022-10-19 02:26:35.676,4100,2RRFTLIglxHgAYQzsSLI4Y,play


In [8]:
sessions_df.dropna(inplace=True)
sessions_df.drop('session_id', axis=1, inplace=True)
sessions_df.drop('event_type', axis=1, inplace=True)
sessions_df

,timestamp,user_id,track_id
0,2022-07-19 17:17:13.000,101,7b7AziuX7EeIzvatDcC0RS
1,2022-07-19 17:20:33.880,101,5RubKOuDoPn5Kj5TLVxSxY
2,2022-07-19 17:22:48.894,101,23zzKOubAzAu8xlQdkVvZ2
3,2022-07-19 17:28:23.521,101,2DVkYfl2YuOMlRfC98srEG
4,2022-07-19 17:32:25.619,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,2022-10-19 02:18:24.943,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,2022-10-19 02:22:52.276,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,2022-10-19 02:23:40.099,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,2022-10-19 02:26:35.676,4100,2RRFTLIglxHgAYQzsSLI4Y


In [9]:
import datetime as dt
sessions_df['timestamp'] = pd.to_datetime(sessions_df['timestamp'])
sessions_df['timestamp'] = (sessions_df['timestamp'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
sessions_df

,timestamp,user_id,track_id
0,1658251033,101,7b7AziuX7EeIzvatDcC0RS
1,1658251233,101,5RubKOuDoPn5Kj5TLVxSxY
2,1658251368,101,23zzKOubAzAu8xlQdkVvZ2
3,1658251703,101,2DVkYfl2YuOMlRfC98srEG
4,1658251945,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,1666145904,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,1666146172,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,1666146220,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,1666146395,4100,2RRFTLIglxHgAYQzsSLI4Y


In [10]:
sessions_df.rename(columns={"timestamp" : "timestamp:float",
                    "user_id":"user_id:token", 
                    "track_id": "item_id:token"}, inplace=True)
sessions_df

,timestamp:float,user_id:token,item_id:token
0,1658251033,101,7b7AziuX7EeIzvatDcC0RS
1,1658251233,101,5RubKOuDoPn5Kj5TLVxSxY
2,1658251368,101,23zzKOubAzAu8xlQdkVvZ2
3,1658251703,101,2DVkYfl2YuOMlRfC98srEG
4,1658251945,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,1666145904,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,1666146172,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,1666146220,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,1666146395,4100,2RRFTLIglxHgAYQzsSLI4Y


In [11]:
sessions_df = sessions_df[
    [
        "user_id:token",
        "item_id:token",
        "timestamp:float"
    ]
]
sessions_df

,user_id:token,item_id:token,timestamp:float
0,101,7b7AziuX7EeIzvatDcC0RS,1658251033
1,101,5RubKOuDoPn5Kj5TLVxSxY,1658251233
2,101,23zzKOubAzAu8xlQdkVvZ2,1658251368
3,101,2DVkYfl2YuOMlRfC98srEG,1658251703
4,101,5wTuWXWREngkFGtBXhOaI9,1658251945
...,...,...,...
1279797,4100,7dWSrvzawBZPa7VVeHZBn7,1666145904
1279798,4100,3tSuRiNgJmMqmYR23dCqnR,1666146172
1279799,4100,3tSuRiNgJmMqmYR23dCqnR,1666146220
1279801,4100,2RRFTLIglxHgAYQzsSLI4Y,1666146395


In [12]:
!mkdir ./sessions
sessions_df[['user_id:token', 'item_id:token', 'timestamp:float']].to_csv("./sessions/sessions.inter", index=False, sep='\t')

del sessions_df
gc.collect()

mkdir: cannot create directory ‘./sessions’: File exists


0

# 2. Trening

In [13]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

from recbole.utils.case_study import full_sort_topk
from recbole.quick_start import objective_function

In [14]:
from ray import tune
from recbole.config import Config
from recbole.data import (
    create_dataset,
    data_preparation,
    save_split_dataloaders,
    load_split_dataloaders,
)
from recbole.data.transform import construct_transform
from recbole.utils import (
    init_logger,
    get_model,
    get_trainer,
    init_seed,
)

In [15]:
parameter_dict = {"data_path": "./",
    "USER_ID_FIELD": "user_id",
    "ITEM_ID_FIELD": "item_id",
    "TIME_FIELD": "timestamp",
    "load_col": {'inter': ['user_id', 'item_id', 'timestamp']},
    "epochs": 10,
    "stopping_step": 3,
    "eval_batch_size": 1024,
    "train_batch_size": 1024,
    "enable_amp": True,
    "MAX_ITEM_LIST_LENGTH": 20,
    "eval_args": {
            'split': {'RS': [8, 1, 1]},
            'group_by': 'user',
            'order': 'TO',
            'mode': 'full'},
    "topk" : 20,
    "valid_metric" : "recall@20",
    "model" : "POP",
    "device": "cuda:0",
    "dataset": "sessions"
}

In [16]:
init_seed(0,True)

In [17]:
from recbole.quick_start import run_recbole

run_recbole(model='Pop', dataset='sessions', config_dict=parameter_dict)

Evaluate   : 100%|█████████████████████| 4000/4000 [00:47<00:00, 84.99it/s, GPU RAM: 0.00 G/14.75 G]


{'best_valid_score': 0.0284,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@20', 0.0284),
              ('mrr@20', 0.0388),
              ('ndcg@20', 0.0279),
              ('hit@20', 0.0685),
              ('precision@20', 0.0246)]),
 'test_result': OrderedDict([('recall@20', 0.0307),
              ('mrr@20', 0.0375),
              ('ndcg@20', 0.0294),
              ('hit@20', 0.0693),
              ('precision@20', 0.0262)])}